In [2]:
import requests

def query_soloartist(search_term):
    """
    Queries Wikidata for entities based on the search term.
    
    Parameters:
    - search_term (str): The exact name or partial name to search for.
    
    Returns:
    - dict: The JSON response from the Wikidata Query Service.
    """
    
    # Define the SPARQL query with a placeholder for the search term
    sparql_query = """
    SELECT ?musician ?musicianLabel ?sitelinks WHERE {
    # Define the professions relevant to solo musicians
    VALUES ?profession {
        wd:Q177220  # Singer
        wd:Q2252262 # rapper
        wd:Q2643890 # vocalist
        wd:Q855091 # Guitarist
        wd:Q486748  # Pianist
        wd:Q386854   # Drummer
        wd:Q1259917  # Violinist
        wd:Q35676   # Cellist
        wd:Q2452838 # Bassist
        wd:Q36927   # Music producer
    }
    
    # Ensure the entity is a human
    ?musician wdt:P31 wd:Q5.
    
    # Ensure the entity has one or more of the relevant professions
    ?musician wdt:P106 ?profession.

    # Search by the exact name of the musician
    ?musician rdfs:label ?musicianLabel.
    FILTER (LANG(?musicianLabel) = "en")
        # for not exact match:
    # FILTER (regex(?entityLabel, "SEARCH_TERM", "i"))
    FILTER (str(?musicianLabel) = "%s")
    
    # Get the number of sitelinks (Wikipedia articles in different languages)
    OPTIONAL { ?musician wikibase:sitelinks ?sitelinks. }
    
    # Retrieve labels in the preferred language
    SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". }
    }
    ORDER BY DESC(?sitelinks)
    LIMIT 5""" % search_term
        
    # Define the URL for the Wikidata Query Service
    url = "https://query.wikidata.org/sparql"
    
    # Define the headers for the request
    headers = {
        "Accept": "application/json"
    }
    
    # Send the request to the Wikidata Query Service
    response = requests.get(url, headers=headers, params={"query": sparql_query, "format": "json"})
    
    # Check if the request was successful
    if response.status_code == 200:
        return response.json()  # Return the JSON response
    else:
        response.raise_for_status()  # Raise an error if the request failed


search_term = "Plini"  # Replace with the search term you want
results = query_soloartist(search_term)
print(results)


{'head': {'vars': ['musician', 'musicianLabel', 'sitelinks']}, 'results': {'bindings': [{'musician': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q56303103'}, 'musicianLabel': {'xml:lang': 'en', 'type': 'literal', 'value': 'Plini'}, 'sitelinks': {'datatype': 'http://www.w3.org/2001/XMLSchema#integer', 'type': 'literal', 'value': '9'}}]}}


In [5]:
def query_musicgroup(search_term):
    """
    Queries Wikidata for entities based on the search term.
    
    Parameters:
    - search_term (str): The exact name or partial name to search for.
    
    Returns:
    - dict: The JSON response from the Wikidata Query Service.
    """
    
    # Define the SPARQL query with a placeholder for the search term
    sparql_query = """SELECT ?group ?groupLabel ?sitelinks WHERE {
    VALUES ?type {
        wd:Q215380   # Musical group
        wd:Q5741069  # Rock group
        wd:Q483501   # Band
        wd:Q868557   # Supergroup
        wd:Q2088357  # Musical ensemble
        wd:Q838948   # Girl group
    }
    
    ?group wdt:P31 ?type.
    ?group rdfs:label ?groupLabel.
    FILTER (LANG(?groupLabel) = "en")
    FILTER (str(?groupLabel) = "%s")
    
    # Get the number of sitelinks (Wikipedia articles)
    OPTIONAL { ?group wikibase:sitelinks ?sitelinks. }
    
    SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". }
    }
    LIMIT 25 """ % search_term
            
    # Define the URL for the Wikidata Query Service
    url = "https://query.wikidata.org/sparql"
    
    # Define the headers for the request
    headers = {
        "Accept": "application/json"
    }
    
    # Send the request to the Wikidata Query Service
    response = requests.get(url, headers=headers, params={"query": sparql_query, "format": "json"})
    
    # Check if the request was successful
    if response.status_code == 200:
        return response.json()  # Return the JSON response
    else:
        response.raise_for_status()  # Raise an error if the request failed



In [6]:
query_musicgroup("U2")


{'head': {'vars': ['group', 'groupLabel', 'sitelinks']},
 'results': {'bindings': [{'group': {'type': 'uri',
     'value': 'http://www.wikidata.org/entity/Q396'},
    'groupLabel': {'xml:lang': 'en', 'type': 'literal', 'value': 'U2'},
    'sitelinks': {'datatype': 'http://www.w3.org/2001/XMLSchema#integer',
     'type': 'literal',
     'value': '115'}}]}}

In [14]:
def query_work(search_term):
    """
    Queries Wikidata for entities based on the search term.
    
    Parameters:
    - search_term (str): The exact name or partial name to search for.
    
    Returns:
    - dict: The JSON response from the Wikidata Query Service.
    """
    
    # Define the SPARQL query with a placeholder for the search term
    sparql_query = """SELECT ?entity ?entityLabel ?sitelinks WHERE {
    VALUES ?type {
        wd:Q7366    # Song
        wd:Q2188189 # Musical work
        wd:Q482994    # Album
        wd:Q134556  # Single
        wd:Q169930  # Extended Play
        wd:Q9734  # Symphony
    }
    
    ?entity wdt:P31 ?type.
    ?entity rdfs:label ?entityLabel.
    FILTER (LANG(?entityLabel) = "en")
    FILTER (str(?entityLabel) = "%s")
    # for not exact match:
    #  FILTER (regex(?entityLabel, "SEARCH_TERM", "i"))
    
    # Get the number of sitelinks (Wikipedia articles)
    OPTIONAL { ?entity wikibase:sitelinks ?sitelinks. }
    
    SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". }
    }
    ORDER BY DESC(?sitelinks)
    LIMIT 5""" % search_term
                
    # Define the URL for the Wikidata Query Service
    url = "https://query.wikidata.org/sparql"
    
    # Define the headers for the request
    headers = {
        "Accept": "application/json"
    }
    
    # Send the request to the Wikidata Query Service
    response = requests.get(url, headers=headers, params={"query": sparql_query, "format": "json"})
    
    # Check if the request was successful
    if response.status_code == 200:
        return response.json()  # Return the JSON response
    else:
        response.raise_for_status()  # Raise an error if the request failed



In [15]:
query_work("Nothing Else Matters")


HTTPError: 500 Server Error: Internal Server Error for url: https://query.wikidata.org/sparql?query=SELECT+%3Fentity+%3FentityLabel+%3Fsitelinks+WHERE+%7B%0A++++VALUES+%3Ftype+%7B%0A++++++++wd%3AQ7366++++%23+Song%0A++++++++wd%3AQ2188189+%23+Musical+work%0A++++++++wd%3AQ482994++++%23+Album%0A++++++++wd%3AQ134556++%23+Single%0A++++++++wd%3AQ169930++%23+Extended+Play%0A++++++++wd%3AQ9734++%23+Symphony%0A++++%7D%0A++++%0A++++%3Fentity+wdt%3AP31+%3Ftype.%0A++++%3Fentity+rdfs%3Alabel+%3FentityLabel.%0A++++FILTER+%28LANG%28%3FentityLabel%29+%3D+%22en%22%29%0A++++FILTER+%28str%28%3FentityLabel%29+%3D+%22Nothing+Else+Matters%22%29%0A++++%23+for+not+exact+match%3A%0A++++%23++FILTER+%28regex%28%3FentityLabel%2C+%22SEARCH_TERM%22%2C+%22i%22%29%29%0A++++%0A++++%23+Get+the+number+of+sitelinks+%28Wikipedia+articles%29%0A++++OPTIONAL+%7B+%3Fentity+wikibase%3Asitelinks+%3Fsitelinks.+%7D%0A++++%0A++++SERVICE+wikibase%3Alabel+%7B+bd%3AserviceParam+wikibase%3Alanguage+%22%5BAUTO_LANGUAGE%5D%2Cen%22.+%7D%0